## Imports

In [10]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, confusion_matrix
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import itertools



## Utils

In [11]:
def generate_hyperparameter_combinations(param_ranges):
    '''
    Parameters:
    param_ranges (dict): Dictionary with hyperparameter names as keys.
                         Each value is a tuple (start, stop, step) indicating the range and step size for the hyperparameter.
    Returns:
    list: List of dictionaries with all possible combinations of hyperparameters.
    '''
    param_values = {
        key: np.arange(start, stop + step, step)
        for key, (start, stop, step) in param_ranges.items()
    }
    
    param_combinations = list(itertools.product(*param_values.values()))
    return [
        dict(zip(param_values.keys(), combination))
        for combination in param_combinations
    ]

## Data load

In [12]:
# One-hot encoding
encoder = OneHotEncoder(categories='auto', sparse_output=False)

# Load training and test files for each dataset from the specified path
monk1_train = pd.read_csv('../Datasets/Monks/monks-1.train', sep='\s+', header=None)
monk1_test = pd.read_csv('../Datasets/Monks/monks-1.test', sep='\s+', header=None)

monk2_train = pd.read_csv('../Datasets/Monks/monks-2.train', sep='\s+', header=None)
monk2_test = pd.read_csv('../Datasets/Monks/monks-2.test', sep='\s+', header=None)

monk3_train = pd.read_csv('../Datasets/Monks/monks-3.train', sep='\s+', header=None)
monk3_test = pd.read_csv('../Datasets/Monks/monks-3.test', sep='\s+', header=None)

# List to store the transformed datasets
monks_train = []
monks_test = []

# Dataset monk1
X1_train = monk1_train.iloc[:, 1:7].values  # Features
y1_train = monk1_train.iloc[:, 0].values    # Labels

X1_test = monk1_test.iloc[:, 1:7].values
y1_test = monk1_test.iloc[:, 0].values

# Apply encoder to monk1
X1_train_encoded = encoder.fit_transform(X1_train)  # Fit and transform on training data
X1_test_encoded = encoder.transform(X1_test)        # Only transform on test data

monks_train.append((X1_train_encoded, y1_train))
monks_test.append((X1_test_encoded, y1_test))

# Dataset monk2
X2_train = monk2_train.iloc[:, 1:7].values
y2_train = monk2_train.iloc[:, 0].values

X2_test = monk2_test.iloc[:, 1:7].values
y2_test = monk2_test.iloc[:, 0].values

# Apply encoder to monk2
X2_train_encoded = encoder.fit_transform(X2_train)
X2_test_encoded = encoder.transform(X2_test)

monks_train.append((X2_train_encoded, y2_train))
monks_test.append((X2_test_encoded, y2_test))

# Dataset monk3
X3_train = monk3_train.iloc[:, 1:7].values
y3_train = monk3_train.iloc[:, 0].values

X3_test = monk3_test.iloc[:, 1:7].values
y3_test = monk3_test.iloc[:, 0].values

# Apply encoder to monk3
X3_train_encoded = encoder.fit_transform(X3_train)
X3_test_encoded = encoder.transform(X3_test)

monks_train.append((X3_train_encoded, y3_train))
monks_test.append((X3_test_encoded, y3_test))


## Model creation

In [13]:
def create_KNN(K=100, metric='euclidean', p = 3, weights='distance'):
    '''
    Create an K-NN model with the parameter K.
    param K: Regularization parameter.
    return: K-NN model.
    '''
    if metric == 'minkowski':
        return KNeighborsClassifier(n_neighbors=K, weights=weights, metric=metric, p=p)
    else:
        return KNeighborsClassifier(n_neighbors=K, weights=weights, metric=metric)

## Double k-fold cross validation

In [14]:
def double_k_fold_cross_validation(data, labels, type = 'euclidean', outer_k=5, inner_k=5, param_grid=None):
    """
    Implements Double K-Fold Cross-Validation

    Parameters
    ---------
     -   data (np.ndarray): Features of the dataset.
     -   labels (np.ndarray): Labels of the dataset.
     -   activ_type (str): Activation function type.
     -   optim_type (str): Optimizer type.
     -   reg_flag (bool): Regularization flag.
     -  in_size (int): Input size.
     -   outer_k (int): Number of folds for outer cross-validation.
     -   inner_k (int): Number of folds for inner cross-validation.
     -   param_grid (list): List of dictionaries with hyperparameters to try.
    
    Returns:
    ----------
     -   list: List of scores obtained for each outer fold.
     -   list: List of best parameters for each outer fold.
    """
    outer_scores = []
    outer_params = []
    
    # Configuration of the outer k-fold cross-validation
    out_kfold = StratifiedKFold(n_splits=outer_k, shuffle=True, random_state=42)

    # Outer cross-validation loop
    out_fold_no = 1
    for train_index, val_index in out_kfold.split(data, labels):
        
        # Split the dataset into training and validation sets for the outer fold
        out_X_train, out_X_val = data[train_index], data[val_index]
        out_y_train, out_y_val = labels[train_index], labels[val_index]
        
        best_params = {}
        best_score = -np.inf

        # Iterate over each set of hyperparameters in the parameter grid
        for params in param_grid:

            inner_scores = []

            # Inner cross-validation loop
            inner_fold_no = 1
            inner_kfold = StratifiedKFold(n_splits=inner_k, shuffle=True, random_state=42)

            for train_index, val_index in inner_kfold.split(out_X_train, out_y_train):
                
                # Split the dataset into training and validation sets for the inner fold
                inner_X_train, inner_X_val = out_X_train[train_index], out_X_train[val_index]
                inner_y_train, inner_y_val = out_y_train[train_index], out_y_train[val_index]

                # Create the SVM model
                model = create_KNN(K=params['K'], metric=type, p=params['P'])
                
                # Train the model on the training set
                model.fit(inner_X_train, inner_y_train)
                
                 # Predict on the validation set
                pred = model.predict(inner_X_val)
                
                # Get the accuracy score for the current fold
                score = accuracy_score(pred, inner_y_val)
                # Append the maximum validation accuracy to inner scores
                inner_scores.append(score)
                inner_fold_no += 1
            
            # Calculate the average score for the current set of hyperparameters
            avg_score = np.mean(inner_scores)

            # Update the best score and parameters if the current average score is better
            if avg_score > best_score:
                best_score = avg_score
                best_params = params

        # Create the SVM model
        model = create_KNN(K=best_params['K'], metric=type, p=best_params['P'])
                
        # Train the model on the training set
        model.fit(out_X_train, out_y_train)
                
        # Predict on the validation set
        pred = model.predict(out_X_val)
                
        # Get the accuracy score for the current fold
        score = accuracy_score(pred, out_y_val)
        # Append the maximum validation accuracy to outer scores
        outer_scores.append(score)
        outer_params.append(best_params)
        out_fold_no += 1
    
    return outer_scores, outer_params

## K-fold cross validation

In [15]:
def k_fold_cross_validation(data, labels, type='rbf', params=None):
    '''
    Perform k-fold cross-validation for SVM.
    
    Parameters:
    -----------
    data : array-like
        Feature data.
    labels : array-like
        Target labels.
    type : str, default='rbf'
        Kernel type for SVM.
    params : dict, optional
        Dictionary of hyperparameters.
    
    Returns:
    --------
    avg_score : float
        Average accuracy score across all folds.
    model : SVC object
        Trained SVM model on the entire dataset.
    '''
    
    # 3. Configure k-fold cross-validation
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # 4. Cross-validation loop
    fold_no = 1
    accuracy_train_per_fold = []  # List to store train accuracy for each fold
    accuracy_per_fold = []  # List to store accuracy for each fold
    for train_index, val_index in kfold.split(data, labels):
        
        # Split the dataset into training and validation sets
        X_train, X_val = data[train_index], data[val_index]
        y_train, y_val = labels[train_index], labels[val_index]

         # Create the SVM model
        model = create_KNN(K=params['K'], metric=type, p=params['P'])

        # Train the model on the training set
        model.fit(X_train, y_train)

        # Predict on the validation set
        pred = model.predict(X_val)
        # Get the accuracy score for the current fold
        score = accuracy_score(pred, y_val)

        train_pred = model.predict(X_train)
        train_score = accuracy_score(train_pred, y_train)

        # Append the accuracy to the list
        accuracy_per_fold.append(score)  # Append the score to the list
        accuracy_train_per_fold.append(train_score)
        fold_no += 1

    # Calculate the average accuracy score across all folds
    avg_score = np.mean(accuracy_per_fold)
    avg_train_score = np.mean(accuracy_train_per_fold)

    # Split the dataset for final training (80% training, 20% validation)
    _, X_val, _, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

     # Create the SVM model
    model = create_KNN(K=params['K'], metric=type, p=params['P'])

    # Train the model on the entire dataset
    model.fit(data, labels)

    return avg_train_score, avg_score, model

## Greed search

In [16]:
def greed_search(data, labels, type='rbf', param_grid=None):
    '''
    Perform greedy search for hyperparameter tuning.
    
    Parameters:
    -----------
    data : array-like
        Feature data.
    labels : array-like
        Target labels.
    type : str, default='rbf'
        Kernel type for SVM.
    param_grid : list of dict, optional
        List of hyperparameter combinations.
    
    Returns:
    --------
    best_scores : list of float
        Best scores obtained during the search.
    best_params_list : list of dict
        Best parameter configurations.
    best_models : list of SVC object
        Best models trained with the best parameter configurations.
    '''
    best_train_scores = []  # List to store the scores
    best_scores = []  # List to store the scores
    best_params_list = []  # List to store the parameter configurations
    best_models = []  # List to store the models
    
    # Iterate over each combination of hyperparameters in the parameter grid
    for params in param_grid:
        # Perform k-fold cross-validation with the current set of hyperparameters
        train_score, score, model = k_fold_cross_validation(data, labels, type, params=params)

        # Add the results to the respective lists
        best_train_scores.append(train_score)
        best_scores.append(score)
        best_params_list.append(params)
        best_models.append(model)

        # Sort the list of scores in descending order and keep only the top 5
        sorted_indices = np.argsort(best_scores)[::-1]  # Get the indices that would sort the scores in descending order
        best_train_scores = [best_train_scores[i] for i in sorted_indices][:5]
        best_scores = [best_scores[i] for i in sorted_indices][:5]  # Keep the top 5 scores
        best_params_list = [best_params_list[i] for i in sorted_indices][:5]  # Keep the top 5 parameter configurations
        best_models = [best_models[i] for i in sorted_indices][:5]  # Keep the top 5 models

    return best_train_scores, best_scores, best_params_list, best_models

## Model selection

In [17]:
def selection(data, labels):
    '''
    Perform hyperparameter selection for SVM models.
    
    Parameters:
    -----------
    data : array-like
        Feature data.
    labels : array-like
        Target labels.
    
    Returns:
    --------
    best_scores : list of float
        Best scores obtained during the search.
    best_params_list : list of dict
        Best parameter configurations.
    best_models : list of SVC object
        Best models trained with the best parameter configurations.
    '''

    # Define the range of hyperparameters
    param_ranges = {
        "K": (1, 75, 1),  # From 1 to 1000 with step of 10
        "P": (3, 10, 1),  # From 1 to 10 with step of 1
    }

    print("Generating hyperparameter combinations...")
    # Generate all combinations of hyperparameters based on the specified ranges
    param_grid = generate_hyperparameter_combinations(param_ranges)

    best_train_scores = []  # List to store the scores
    best_scores = []  # List to store the scores
    best_params_list = []  # List to store the parameter configurations
    best_models = []  # List to store the models
    best_types = []  # List to store the kernel types

    # Define the kernel types to be tested
    types = ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
    for type in types:
        # Perform greedy search for each kernel type
        actual_train_scores, actual_scores, actual_params_list, actual_models = greed_search(data, labels, type, param_grid)

        # Extend the lists with the results from the current kernel type
        best_train_scores.extend(actual_train_scores)
        best_scores.extend(actual_scores)
        best_params_list.extend(actual_params_list)
        best_models.extend(actual_models)
        best_types.extend([type] * len(actual_scores))

        # Sort the scores in descending order and keep only the top 5
        sorted_indices = np.argsort(best_scores)[::-1]
        best_train_scores = [best_train_scores[i] for i in sorted_indices][:5]
        best_scores = [best_scores[i] for i in sorted_indices][:5]
        best_params_list = [best_params_list[i] for i in sorted_indices][:5]
        best_models = [best_models[i] for i in sorted_indices][:5]
        best_types = [best_types[i] for i in sorted_indices][:5]

    # Print the best scores, kernel types, and parameter configurations
    for i in range(len(best_scores)):
        print(f"Score: {best_scores[i]}, train score: {best_train_scores[i]}, type: {best_types[i]}, parameters: {best_params_list[i]}")

    return best_train_scores, best_scores, best_params_list, best_models



In [18]:
print("-----------------MONK1-----------------")
# Perform hyperparameter selection for the MONK1 dataset and get the best models
_, _, best_param_list_monk1, best_models_monk_1 = selection(monks_train[0][0], monks_train[0][1])

print("-----------------MONK2-----------------")
# Perform hyperparameter selection for the MONK2 dataset and get the best models
_, _, best_param_list_monk2, best_models_monk_2 = selection(monks_train[1][0], monks_train[1][1])

print("-----------------MONK3-----------------")
# Perform hyperparameter selection for the MONK3 dataset and get the best models
_, _, best_param_list_monk3, best_models_monk_3 = selection(monks_train[2][0], monks_train[2][1])

-----------------MONK1-----------------
Generating hyperparameter combinations...
Score: 0.7833333333333334, train score: 1.0, type: manhattan, parameters: {'K': 8, 'P': 5}
Score: 0.7833333333333334, train score: 1.0, type: manhattan, parameters: {'K': 8, 'P': 3}
Score: 0.7833333333333334, train score: 1.0, type: manhattan, parameters: {'K': 8, 'P': 4}
Score: 0.7833333333333334, train score: 1.0, type: manhattan, parameters: {'K': 8, 'P': 6}
Score: 0.7833333333333334, train score: 1.0, type: manhattan, parameters: {'K': 8, 'P': 10}
-----------------MONK2-----------------
Generating hyperparameter combinations...
Score: 0.6452205882352942, train score: 1.0, type: euclidean, parameters: {'K': 26, 'P': 10}
Score: 0.6452205882352942, train score: 1.0, type: euclidean, parameters: {'K': 26, 'P': 6}
Score: 0.6452205882352942, train score: 1.0, type: euclidean, parameters: {'K': 26, 'P': 4}
Score: 0.6452205882352942, train score: 1.0, type: euclidean, parameters: {'K': 26, 'P': 3}
Score: 0.64

In [20]:
# Generate all combinations of hyperparameters based on the specified ranges
param_grid_final_monk1 = [best_param_list_monk1[0]]

scores_monk1, _ = double_k_fold_cross_validation(monks_train[0][0], monks_train[0][1], type='manhattan', outer_k=5, inner_k=5, param_grid=param_grid_final_monk1)

variance_1 = np.var(scores_monk1)
mean_monk1 = np.mean(scores_monk1)

print(f"Mean MONK 1: {mean_monk1}")
print(f"Variance MONK 1: {variance_1}")

Mean MONK 1: 0.734
Variance MONK 1: 0.006064


In [21]:
# Generate all combinations of hyperparameters based on the specified ranges
param_grid_final_monk2 = [best_param_list_monk1[1]]

scores_monk2, _ = double_k_fold_cross_validation(monks_train[1][0], monks_train[1][1], type='euclidean', outer_k=5, inner_k=5, param_grid=param_grid_final_monk2)
variance_2 = np.var(scores_monk2)
mean_monk2 = np.mean(scores_monk2)

print(f"Mean MONK 2: {mean_monk2}")
print(f"Variance MONK 2: {variance_2}")

Mean MONK 2: 0.6151515151515151
Variance MONK 2: 0.0005177284007104704


In [22]:
# Generate all combinations of hyperparameters based on the specified ranges
param_grid_final_monk3 = [best_param_list_monk3[2]]

scores_monk3, _ = double_k_fold_cross_validation(monks_train[2][0], monks_train[2][1], type='minkowski', outer_k=5, inner_k=5, param_grid=param_grid_final_monk3)

variance_3 = np.var(scores_monk3)
mean_monk3 = np.mean(scores_monk3)

print(f"Mean MONK 3: {mean_monk3}")
print(f"Variance MONK 3: {variance_3}")

Mean MONK 3: 0.877
Variance MONK 3: 0.00278377777777778


## Model assessment

In [23]:
# 5. Model evaluation
y1_pred = best_models_monk_1[0].predict(X1_test_encoded)

# 6. Report the results
print("Accuracy:", accuracy_score(y1_test, y1_pred))
print("\nClassification Report:\n", classification_report(y1_test, y1_pred))

Accuracy: 0.8402777777777778

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.90      0.85       216
           1       0.88      0.78      0.83       216

    accuracy                           0.84       432
   macro avg       0.84      0.84      0.84       432
weighted avg       0.84      0.84      0.84       432



In [24]:
# 5. Model evaluation
y2_pred = best_models_monk_2[0].predict(X2_test_encoded)

# 6. Report the results
print("Accuracy:", accuracy_score(y2_test, y2_pred))
print("\nClassification Report:\n", classification_report(y2_test, y2_pred))

Accuracy: 0.7939814814814815

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.93      0.86       290
           1       0.78      0.51      0.62       142

    accuracy                           0.79       432
   macro avg       0.79      0.72      0.74       432
weighted avg       0.79      0.79      0.78       432



In [25]:
# Predict on the test set for MONK3
y3_pred = best_models_monk_3[0].predict(X3_test_encoded)

# Report the results
print("Accuracy:", accuracy_score(y3_test, y3_pred))
print("\nClassification Report:\n", classification_report(y3_test, y3_pred))

Accuracy: 0.9305555555555556

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93       204
           1       0.98      0.89      0.93       228

    accuracy                           0.93       432
   macro avg       0.93      0.93      0.93       432
weighted avg       0.93      0.93      0.93       432

